# Capstone project for applied Data Science (IBM professional certification on Coursera) - PART 3

## By Laura-June Clarke, July 2020

## PART 3: Explore and cluster the neighborhoods in Toronto. 

In [1]:
# import required libraries and data

#libto handle JSON files
import json

# lb to handle data in vectorized manner
import numpy as np

#lib for data analyse
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#!conda install -c conda-forge geopy --yes # uncomment this line if needed
! pip install geopy

# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim

# lib to handle requess
import requests 

from bs4 import BeautifulSoup
import xml

# import k-means from clustering stage
from sklearn.cluster import KMeans
import os

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if needed
! pip install folium
# map rendering library
import folium 

# import k-means from clustering stage
import matplotlib.cm as cm
import matplotlib.colors as colors

# get data from URL and download it
website = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
url = requests.get(website).text
CA_data = BeautifulSoup(url, 'xml')
table=CA_data.find('table')

print ("libraries and data downloaded!")

libraries and data downloaded!


In [2]:
# Create dataframe and update with the information for Wikipedia

column_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)

for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data
        
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [3]:
df=df[df['Borough']!='Not assigned']

dummy_df=df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
dummy_df=dummy_df.reset_index(drop=False)
dummy_df.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)

df_merge = pd.merge(df, dummy_df, on='Postalcode')

df_merge.drop(['Neighborhood'],axis=1,inplace=True)

df_merge.drop_duplicates(inplace=True)

df_merge.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)

df_merge.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [4]:
#Get coordinates

def get_geocode(postal_code):

    lat_lng_coords = None # initialize your variable to None
    
    while(lat_lng_coords is None): # loop until you get the coordinates
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    
    return latitude,longitude

In [5]:
geo_df=pd.read_csv('http://cocl.us/Geospatial_data')
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [6]:
geo_df.rename(columns={'Postal Code':'Postalcode'},inplace=True)
geo_merged = pd.merge(geo_df, df_merge, on='Postalcode')

geo_data=geo_merged[['Postalcode','Borough','Neighborhood','Latitude','Longitude']]

toronto_data=geo_data[geo_data['Borough'].str.contains("Toronto")]
toronto_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


#### Explore and cluster the neighborhoods in Toronto. 

In [7]:
# Foursquare API creditentials
CLIENT_ID = 'HEZLAWN4GXHKE2YLMXWMQTRNKXBBY0R4FMQOOZQLEVIHIQMT' # your Foursquare ID
CLIENT_SECRET = '31ETZTQIRS0PHXHZQ0VJ1TCDOB1300IFFGSJKWDATCM2LD0M' # your Foursquare Secret
VERSION = '20180604'

LIMIT = 100
radius = 500
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HEZLAWN4GXHKE2YLMXWMQTRNKXBBY0R4FMQOOZQLEVIHIQMT
CLIENT_SECRET:31ETZTQIRS0PHXHZQ0VJ1TCDOB1300IFFGSJKWDATCM2LD0M


In [ ]:
#Create function to explore venue around coordinates

def getNearbyVenues(names, latitudes, longitudes):
    #radius=500 #within 500 metres
    #LIMIT=100 #limited to 100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        #GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        #Keep only columns that include venue name, and anything that is associated with location
        venues_list.append([(
            name, 
            lat, 
            lng, 
            elements['venue']['name'], 
            elements['venue']['location']['lat'], 
            elements['venue']['location']['lng'],  
            elements['venue']['categories'][0]['name']) for elements in results])

    #List nearby venues and keep revelant information
    v_near = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    v_near.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(v_near)

In [ ]:
#clean dataframe
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )
toronto_venues.head()

In [ ]:
toronto_venues.groupby('Neighborhood').count()

#### We use One Hot encoding for the clustering

In [ ]:
# one hot encoding
toronto_oh = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_oh.drop(['Neighborhood'],axis=1,inplace=True) 
toronto_oh.insert(loc=0, column='Neighborhood', value=toronto_venues['Neighborhood'] )

In [ ]:
toronto_oh.shape
toronto_grouped = toronto_oh.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns with number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_filtered = pd.DataFrame(columns=columns)
neighborhoods_venues_filtered['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_filtered.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_filtered.head()

In [ ]:
#Precise the number of clusters
k = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=k, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

In [ ]:
neighborhoods_venues_filtered.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_filtered.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

In [ ]:
neighborhoods_venues_filtered.head()

In [ ]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Manhattan s coordinates are {}, {}.'.format(latitude, longitude))

In [ ]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters